In [1]:
"""
SHERPA is a Python library for hyperparameter tuning of machine learning models.
Copyright (C) 2018  Lars Hertel, Peter Sadowski, and Julian Collado.

This file is part of SHERPA.

SHERPA is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

SHERPA is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with SHERPA.  If not, see <http://www.gnu.org/licenses/>.
"""
import sherpa
import sherpa.algorithms.bayesian_optimization as bayesian_optimization
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.datasets import mnist
from keras.optimizers import Adam

 /Users/larshertel/python3env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning:Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
Using TensorFlow backend.


### Dataset Preparation

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

### Sherpa Setup

In [7]:
parameters = [sherpa.Continuous('learning_rate', [1e-4, 1e-2], 'log'),
              sherpa.Discrete('num_units', [32, 128]),
              sherpa.Choice('activation', ['relu', 'tanh', 'sigmoid'])]
algorithm = alg = sherpa.algorithms.SuccessiveHalving(r=1, R=9, eta=3, s=0, max_finished_configs=5)
study = sherpa.Study(parameters=parameters,
                     algorithm=algorithm,
                     lower_is_better=False)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://169.234.54.206:8880 if on a cluster or
http://localhost:8880 if running locally.
-------------------------------------------------------


### Hyperparameter Optimization

In [8]:
epochs = 5
model_dict = {}

for trial in study:
    initial_epoch = {1: 0, 3: 1, 9: 4}[trial.parameters['resource']]
    epochs = trial.parameters['resource'] + initial_epoch
    
    print(f"Training Trial {trial.id} for resource {trial.parameters['resource']}")
    print(f"Loading model from: ", trial.parameters['load_from'])
    print(f"Storing model as: ", trial.parameters['save_to'])
    
    
    if trial.parameters['load_from'] not in model_dict:
        lr = trial.parameters['learning_rate']
        num_units = trial.parameters['num_units']
        act = trial.parameters['activation']

        # Create model
        model = Sequential([Flatten(input_shape=(28, 28)),
                            Dense(num_units, activation=act),
                            Dense(10, activation='softmax')])
        optimizer = Adam(lr=lr)
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        model_dict[trial.parameters['save_to']] = model
        

    # Train model
    for i in range(initial_epoch, epochs):
        model.fit(x_train, y_train, initial_epoch=i, epochs=i+1)
        loss, accuracy = model.evaluate(x_test, y_test)
        study.add_observation(trial=trial, iteration=i,
                              objective=accuracy,
                              context={'loss': loss})
    study.finalize(trial=trial)

Training Trial 1 for resource 1
Loading model from:  
Storing model as:  1
Epoch 1/1
10000/10000 [==============================] - 0s 30us/step
Training Trial 2 for resource 1
Loading model from:  
Storing model as:  2
Epoch 1/1
10000/10000 [==============================] - 0s 30us/step
Training Trial 3 for resource 1
Loading model from:  
Storing model as:  3
Epoch 1/1
10000/10000 [==============================] - 0s 31us/step
Training Trial 4 for resource 3
Loading model from:  1
Storing model as:  4
Epoch 2/2
10000/10000 [==============================] - 0s 17us/step
Epoch 3/3
10000/10000 [==============================] - 0s 18us/step
Epoch 4/4
10000/10000 [==============================] - 0s 17us/step
Training Trial 5 for resource 1
Loading model from:  
Storing model as:  5
Epoch 1/1
10000/10000 [==============================] - 0s 32us/step
Training Trial 6 for resource 1
Loading model from:  
Storing model as:  6
Epoch 1/1
10000/10000 [==============================] - 0s

KeyboardInterrupt: 